In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from implicit.als import AlternatingLeastSquares
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix

In [3]:
data = pd.read_csv("C:\\Users\\shriv\\Downloads\\Telegram Desktop\\Final merged.csv")
user_col = 'reviewerID'
item_col = 'asin'
rating_col = 'Rating'
title_col = 'title'

In [4]:
print("Number of rows in the dataset:", len(data))

Number of rows in the dataset: 49999


In [5]:
user_to_index = {user_id: index for index, user_id in enumerate(data[user_col].unique())}
item_to_index = {item_id: index for index, item_id in enumerate(data[item_col].unique())}
data['user_index'] = data[user_col].map(user_to_index)
data['item_index'] = data[item_col].map(item_to_index)

In [6]:
interaction_matrix = coo_matrix((data[rating_col], (data['user_index'], data['item_index'])))

In [7]:
# Specify the column with null values
column_with_nulls = 'Rating'  # Adjust with the column name containing nulls

# Remove rows with null values in the specified column
data = data.dropna(subset=[column_with_nulls])


In [8]:
data.isnull().sum()

asin           0
Rating         0
reviewText     0
reviewerID     0
summary        0
verified       0
brand          0
main_cat       0
price          0
title          0
description    0
user_index     0
item_index     0
dtype: int64

In [9]:
model = AlternatingLeastSquares(factors=10, regularization=0.01, iterations=50)
model.fit(interaction_matrix.T)

C:\Users\shriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0010349750518798828 seconds
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

In [10]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
text_features = tfidf_vectorizer.fit_transform(data['reviewText'] + ' ' + data['summary'])


In [11]:
item_index_to_id = {index: item_id for item_id, index in item_to_index.items()}
item_index_to_title = {index: title for index, title in zip(data['item_index'], data[title_col])}

In [12]:
def user_has_few_interactions(user_index, threshold):
    user_interactions = interaction_matrix.getrow(user_index).nnz
    return user_interactions < threshold


In [13]:
def recommend_content_based(user_index, text_features, num_recommendations):
    user_profile = text_features[user_index]
    item_similarities = cosine_similarity(user_profile, text_features).flatten()
    top_item_indices = item_similarities.argsort()[::-1]

    user_interacted_items = set(data[data['user_index'] == user_index]['item_index'].values)
    recommended_item_indices_in_text_features = [index for index in top_item_indices if index not in user_interacted_items][:num_recommendations]

    return recommended_item_indices_in_text_features


In [14]:
def recommend_collaborative(user_index, model, num_recommendations):
    user_items = interaction_matrix.tocsr()
    recommended_items = model.recommend(user_index, user_items, N=num_recommendations)
    recommended_item_ids = [item_index_to_id[item_index] for item_index, _ in recommended_items]

    print("Recommended item IDs:", recommended_item_ids)

    return recommended_item_ids

In [15]:
def hybrid_recommendation(user_id, num_recommendations):
    user_index = user_to_index.get(user_id)
    
    print("User ID:", user_id)
    print("User Index:", user_index)

    if user_index is None:
        print("User not found.")
        return []

    if user_has_few_interactions(user_index, threshold=10):
        print("Using content-based recommendation.")
        return recommend_content_based(user_index, text_features, num_recommendations)
    else:
        print("Using collaborative recommendation.")
        return recommend_collaborative(user_index, model, num_recommendations)

In [19]:
np.random.seed(42)
ground_truth_items = np.random.choice(data[item_col].unique(), size=10, replace=False)

In [28]:
# Example user ID and number of recommendations
user_id_to_recommend = 'A1BY99OOCM9Q0K'
num_recommendations = 10

# Get hybrid recommendations
recommendations = hybrid_recommendation(user_id_to_recommend, num_recommendations)
print("Hybrid Recommendations:", recommendations)

# Print the recommended titles
# Print the recommended titles





User ID: A1BY99OOCM9Q0K
User Index: 36840
Using content-based recommendation.
Hybrid Recommendations: [36840, 3910, 42688, 24513, 39566, 45171, 6023, 6005, 21900, 2652]


In [27]:
def calculate_metrics(true_positives, false_positives, false_negatives):
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

In [23]:
def evaluate_recommendations(recommended_items, ground_truth_items):
    true_positives = len(set(recommended_items) & set(ground_truth_items))
    false_positives = len(set(recommended_items) - set(ground_truth_items))
    false_negatives = len(set(ground_truth_items) - set(recommended_items))

    precision, recall, f1 = calculate_metrics(true_positives, false_positives, false_negatives)
    return precision, recall, f1

In [24]:
precision, recall, f1 = evaluate_recommendations(recommendations, ground_truth_items)

In [17]:
for index in recommendations:
    val = data.iloc[index]
    print(val.title)
    print(val.asin)

Red Heart Super Saver Yarn, Hunter Green
B000Y00DDK
Aveeno Protect + Hydrate Face Moisturizing Sunscreen Lotion with Broad Spectrum SPF 50 &amp; Antioxidant Oat, Oil-Free, Lightweight, Sweat- &amp; Water-Resistant Sun Protection, Travel-Size, 3 oz
B00NQ2P4OQ
The First Years GumDrop Newborn Pacifier
B01ABP1HLE
Lion Brand Yarn Pound of Love Bubblegum, Bubble Gum
B0001DS5MQ
Shea Moisture - Olive &amp; Marula Baby Head-to-Toe Wash &amp; Shampoo with Avocado &amp; Shea Butter - 13 oz.
B00A2WF0D8
VELCRO Brand - Industrial Strength | Indoor &amp; Outdoor Use | Superior Holding Power on Smooth Surfaces | Size 4ft x 2in | Tape, Black - Pack of 1
B00006IC2T
Braun Clean &amp; Renew Refill Cartridges CCR - 2 Count (Packaging May Vary)
B000050FDY
Braun Clean &amp; Renew Refill Cartridges CCR - 2 Count (Packaging May Vary)
B000050FDY
Johnson's Baby Oil, Mineral Oil Enriched with Aloe Vera and Vitamin E, 14 fl. oz
B01EMZ76IW
Aquaphor Healing Ointment,Advanced Therapy Skin Protectant 14 Ounce ( Pack M

In [ ]:
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)